<a href="https://colab.research.google.com/github/mithunkumarsr/CodeMixingDravidianLanguage/blob/main/CS_TamilTranslate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install spellchecker
!pip install indexer
from google.cloud import translate
from google.cloud import language
#from spellchecker import SpellChecker
from textblob import TextBlob

  Using cached indexer-0.6.2.tar.gz (14 kB)
ERROR: Could not find a version that satisfies the requirement indexer (from versions: 0.6.2)
ERROR: No matching distribution found for indexer


In [ ]:
import pandas as pd
data = pd.read_csv('/content/tamil_sentiment_full_test_withtlabels.tsv', error_bad_lines=False)

b'Skipping line 8: expected 1 fields, saw 7\nSkipping line 39: expected 1 fields, saw 3\nSkipping line 49: expected 1 fields, saw 7\nSkipping line 50: expected 1 fields, saw 2\nSkipping line 65: expected 1 fields, saw 3\nSkipping line 82: expected 1 fields, saw 3\nSkipping line 91: expected 1 fields, saw 3\nSkipping line 105: expected 1 fields, saw 2\nSkipping line 106: expected 1 fields, saw 2\nSkipping line 119: expected 1 fields, saw 2\nSkipping line 136: expected 1 fields, saw 2\nSkipping line 140: expected 1 fields, saw 2\nSkipping line 141: expected 1 fields, saw 2\nSkipping line 157: expected 1 fields, saw 2\nSkipping line 165: expected 1 fields, saw 2\nSkipping line 171: expected 1 fields, saw 2\nSkipping line 176: expected 1 fields, saw 4\nSkipping line 182: expected 1 fields, saw 2\nSkipping line 195: expected 1 fields, saw 3\nSkipping line 200: expected 1 fields, saw 3\nSkipping line 243: expected 1 fields, saw 2\nSkipping line 260: expected 1 fields, saw 2\nSkipping line 26

In [ ]:
data

,id\ttext\tcategory
0,Tam_1\tவீர செங்குந்தர் சார்பாக இந்த திரைப்படம்...
1,Tam_2\tTeruk ah irukku .... mokke movie .. was...
2,Tam_3\tmanitha samuthaayam amaipil irunthu int...
3,Tam_4\tJJ mam we miss u\tPositive
4,Tam_5\tSubtitle me traller dekhne wale like ka...
...,...
4059,Tam_4398\tIthukum dislike potta kammanattti ko...
4060,Tam_4399\tSuyama Sinthikiravan than super Hero...
4061,Tam_4400\tSuper thalaiva.... Nee mass dha eppa...
4062,Tam_4401\tபெண்ணை அடிமையாக்க நினைக்கும் இந்த பட...


In [ ]:
import os
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file("/content/cs-language-e8f4fa6845c9.json")
client = language.LanguageServiceClient(credentials=credentials)

GOOGLE_APPLICATION_CREDENTIALS='/content/cs-language-e8f4fa6845c9.json'
def goog_trans_tamil_to_english(token):

    translate_client = translate.Client()
    translation = translate_client.translate(token, source_language='ta')
    converted_tok = translation['translatedText']
    return converted_tok


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/cs-language-e8f4fa6845c9.json"

client = language.LanguageServiceClient.from_service_account_json("/content/cs-language-e8f4fa6845c9.json")

goog_trans_tamil_to_english('என்னடா இது சிஸ்டத்தை வச்சு செய்ஞ்சுட்டானுங்க')
goog_trans_tamil_to_english('நாடார் சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்')
goog_trans_tamil_to_english('தூத்துக்குடி நாடார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்.')
goog_trans_tamil_to_english('Yebbah Dei ... Nalla Padatha Keduthutingale da ... Go and Watch Pink')
goog_trans_tamil_to_english('இந்த title காக வே படம் பார்க்கணும் theater ல போய்....    ')

'Go to the theater to watch the movie Way for this title ....'

In [ ]:
def get_sub(token):

    testimonial = TextBlob(token)
    theory = testimonial.sentiment.subjectivity
    return theory


In [ ]:
def spell_correct(token):

    spell = SpellChecker()

    if(spell.known(token)):
        return token
    else:
        spell_correct_tok = spell.correction(token)
    return spell_correct_tok

In [ ]:
get_sub(goog_trans_tamil_to_english('தூத்துக்குடி நாடார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்.'))
get_sub('தூத்துக்குடி நாடார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்.')

0.0

In [ ]:
import csv
import json



def get_neu_set_cc():

    neu_features = "./feature_files/neu_features.txt"
    neu_feature_file = open(neu_features, "r")

    neu_set_cc = set()

    for neu_word in neu_feature_file:

        neu_word = neu_word.lower()
        neu_set_cc.add(neu_word)

    return neu_set_cc


def get_neu_set_c():

    neu_features = "./feature_files/neu_features.txt"
    neu_feature_file = open(neu_features, "r")

    neu_set_c = set()

    for neu_word in neu_feature_file:

        neu_word = neu_word.lower()
        neu_set_c.add(neu_word)

    return neu_set_c


def get_neu_set():

    neu_features = "./feature_files/neu_features.txt"
    neu_feature_file = open(neu_features, "r")

    neu_set = set()

    for neu_word in neu_feature_file:

        neu_word = neu_word.lower()
        neu_set.add(neu_word)
        
    neg_set_cc = get_neg_set_cc()
    neu_set.difference_update(neg_set_cc)

    pos_set_cc = get_pos_set_cc()
    neu_set.difference_update(pos_set_cc)

    return neu_set


def get_neg_set_cc():

    neg_features = "./feature_files/neg_features.txt"
    neg_feature_file = open(neg_features, "r")

    neg_set_cc = set()

    for neg_word in neg_feature_file:

        neg_word = neg_word.lower()
        neg_set_cc.add(neg_word)

    return neg_set_cc


def get_neg_set_c():

    neg_features = "./feature_files/neg_features.txt"
    neg_feature_file = open(neg_features, "r")

    neg_set_c = set()

    for neg_word in neg_feature_file:

        neg_word = neg_word.lower()
        neg_set_c.add(neg_word)

    return neg_set_c


def get_neg_set():

    neg_features = "./feature_files/neg_features.txt"
    neg_feature_file = open(neg_features, "r")

    neg_set = set()

    for neg_word in neg_feature_file:

        neg_word = neg_word.lower()
        neg_set.add(neg_word)

    pos_set_c = get_pos_set_c()
    neg_set.difference_update(pos_set_c)

    neu_set_cc = get_neu_set_cc()
    neg_set.difference_update(neu_set_cc)

    return neg_set


def get_pos_set_cc():

    pos_features = "./feature_files/pos_features.txt"
    pos_feature_file = open(pos_features, "r")

    pos_set_cc = set()

    for pos_word in pos_feature_file:

        pos_word = pos_word.lower()
        pos_set_cc.add(pos_word)

    return pos_set_cc


def get_pos_set_c():

    pos_features = "./feature_files/pos_features.txt"
    pos_feature_file = open(pos_features, "r")

    pos_set_c = set()

    for pos_word in pos_feature_file:

        pos_word = pos_word.lower()
        pos_set_c.add(pos_word)

    return pos_set_c


def get_pos_set():

    pos_features = "./feature_files/pos_features.txt"
    pos_feature_file = open(pos_features, "r")

    pos_set = set()

    for pos_word in pos_feature_file:

        pos_word = pos_word.lower()
        pos_set.add(pos_word)

    neg_set_c = get_neg_set_c()
    pos_set.difference_update(neg_set_c)

    neu_set_c = get_neu_set_c()
    pos_set.difference_update(neu_set_c)

    return pos_set